In [1]:
%reset -sf

In [2]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
# d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [56]:
# trr = ""
# srr = """
# """.strip()

with open("/Users/hkmac/Downloads/input.txt") as f:
    srr = f.read().strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 13
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 5
mul y x
add z y
""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 14
mul y x
add z y
""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 15
mul y x
add z y""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 11
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 16
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -16
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 8
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -11
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 9
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -6
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 2
mul y x
add z y
""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 11
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 13
mul y x
add z y
""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 10
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 16
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -10
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 6
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -8
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 6
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -11
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 9
mul y x
add z y""".strip()

srr = """
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 5
mul y x
add z y""".strip()

with open("/Users/hkmac/Downloads/input.txt") as f:
    srr = f.read().strip()


# srr = """
# """.strip()

# srr = """
# """.strip()

# srr = """
# """.strip()

# srr = """
# """.strip()

# srr = """
# """.strip()

# srr = """
# """.strip()

# a,b = srr.split("\n\n")
arr = srr.split("\n")

In [67]:
def test(vrr):
    def print(*args):
        return
#     assert len(vrr) == 14
    vrr = [int(x) for x in vrr]
    vrr = vrr[::-1]
    
    variables = [0,0,0,0]
    
    for row in arr:
        op, *args = row.split()
#         print(variables, row)

        str_to_var = {"w": 0, "x": 1, "y": 2, "z": 3}
        
        if op == "inp":
            a = str_to_var[args[0]]
            variables[a] = vrr.pop()
            print(variables)
            continue
            
        a = str_to_var[args[0]]
        if args[1] in str_to_var:
            bval = variables[str_to_var[args[1]]]
        else:
            bval = int(args[1])
            
#         print(op, a, bval)
        
        if op == "add":
            variables[a] += bval
            continue
        
        if op == "mul":
            # Multiply the value of a by the value of b, then store the result in variable a.
            variables[a] *= bval
            continue
        
        if not variables[a] >= 0 and bval > 0:
            return False

        if op == "div":
            # Divide the value of a by the value of b, 
            # truncate the result to an integer, then store the result in variable a.
            variables[a] = variables[a] // bval
        
        if op == "mod":
            # Divide the value of a by the value of b, 
            # truncate the result to an integer, then store the result in variable a.
            variables[a] = variables[a] % bval
            
        if op == "eql":
            # If the value of a and b are equal, then store the value 1 in variable a.
            # Otherwise, store the value 0 in variable a.
            variables[a] = int(variables[a] == bval)
            
    assert not vrr
    print("end")
    return variables[-1] == 0
    return True

In [68]:
test("99998999999995")

False

In [69]:
# for comb in itertools.product("987654321", repeat=14):
#     if test(comb):
#         print(comb, "ok")
#         break

In [70]:
# comb

In [ ]:
# test("9"*14)

In [ ]:
# 999

In [ ]:
# 9899

In [ ]:
# 9

In [ ]:
# 9999

In [ ]:
# 99

In [ ]:
# 999 9899 9 9999 95

In [66]:
# ('9', '9', '9', '9', '9', '8', '6', '1', '3', '5', '3', '8', '3', '8')